## Import necessary libraries

In [ ]:
import openfermion
import openfermionpyscf
from openfermion.transforms import jordan_wigner, get_fermion_operator

import os
import timeit

import matplotlib.pyplot as plt
from scipy.optimize import minimize
import numpy as np

from itertools import product 

## Define equilibrium geometries and basis for your target molecule (here, it's water)

In [62]:
geometry = [('O', (0.1173, 0.0, 0.0)), ('H', (-0.4691, 0.7570, 0.0)),
            ('H', (-0.4691, -0.7570, 0.0))]
basis = 'sto3g' #let's use a very simple basis set to keep the number of qubits small
multiplicity = 1
charge = 0

## Perform a simple fci calculation (i also did scf and cisd but fci is probably best)

In [63]:
#solve the Hartree-Fock equations
molecule = openfermionpyscf.run_pyscf(openfermion.MolecularData(geometry, basis, multiplicity, charge), run_fci=True, run_scf=True, run_cisd=True)

## Extract the 2-rdm

In [64]:
two_rdm = molecule.fci_two_rdm
print(two_rdm)

[[[[ 1.99999390e+00 -3.76433777e-05 -3.42927267e-20 ... -9.53652709e-34
    -6.22734999e-05  9.05852885e-18]
   [-3.76433777e-05  5.56726792e-05 -4.38280334e-20 ... -5.53874657e-34
    -1.90104500e-06  2.95132197e-19]
   [-3.42927267e-20 -4.38280334e-20  1.21481626e-04 ... -2.57056221e-20
    -3.12032170e-19 -2.42069136e-06]
   ...
   [-9.53652709e-34 -5.53874657e-34 -2.57056221e-20 ...  4.35946723e-05
     2.09982568e-34  4.18807865e-20]
   [-6.22734999e-05 -1.90104500e-06 -3.12032170e-19 ...  2.09982568e-34
    -1.28313983e-03  2.16263576e-18]
   [ 9.05852885e-18  2.95132197e-19 -2.42069136e-06 ...  4.18807865e-20
     2.16263576e-18 -8.42317143e-04]]

  [[-3.76433777e-05 -1.99211234e+00 -1.33495888e-17 ... -1.03420500e-31
    -1.32255273e-03 -5.72005147e-16]
   [ 3.98422831e+00  3.63556399e-06 -1.01019340e-19 ... -2.09085756e-33
    -1.21005485e-04  1.84869973e-17]
   [ 2.67012045e-17  3.19677354e-21  1.36962699e-04 ... -5.93139785e-20
    -8.15856308e-20 -5.73631370e-06]
   ...
   

## Extract the 1-rdm

In [65]:
one_rdm = molecule.fci_one_rdm # 1-RDM: shape (n_orbitals, n_orbitals)
print(one_rdm)

[[ 1.99999635e+00 -3.86210001e-05 -3.44600004e-20 -3.72357121e-05
  -9.57654908e-34 -6.22681345e-05  9.05458103e-18]
 [-3.86210001e-05  1.99211719e+00  1.33503245e-17  9.37106617e-03
   1.03430117e-31  1.32263493e-03  5.72011287e-16]
 [-3.44600004e-20  1.33503245e-17  1.97403668e+00 -1.75333996e-18
   1.18124230e-17 -4.37446962e-16 -3.88972899e-03]
 [-3.72357121e-05  9.37106617e-03 -1.75333996e-18  1.98263086e+00
  -1.80744589e-31  2.32786920e-02 -1.02303136e-15]
 [-9.57654908e-34  1.03430117e-31  1.18124230e-17 -1.80744589e-31
   1.99832548e+00  3.84193050e-31  8.97132042e-17]
 [-6.22681345e-05  1.32263493e-03 -4.37446962e-16  2.32786920e-02
   3.84193050e-31  2.64011237e-02 -2.48149895e-17]
 [ 9.05458103e-18  5.72011287e-16 -3.88972899e-03 -1.02303136e-15
   8.97132042e-17 -2.48149895e-17  2.64923106e-02]]


##  von-neuman entropy calculation function

In [66]:
import numpy as np

def von_neumann_entropy(rdm, base=2, tol=1e-12):
    """Compute von Neumann entropy of a density matrix.
    
    Args:
        rdm (ndarray): A square density matrix.
        base (float): Logarithmic base (e.g., np.e or 2).
        tol (float): Tolerance for filtering small eigenvalues.
    
    Returns:
        float: Entropy S = -Tr(ρ log ρ)
    """
    # Ensure Hermitian
    rdm = (rdm + rdm.conj().T) / 2
    eigvals = np.linalg.eigvalsh(rdm)
    eigvals = eigvals[eigvals > tol]  # Filter numerical noise
    entropy = -np.sum(eigvals * np.log(eigvals)) / np.log(base)
    return entropy


##### I need to understand better what data is contained in the 1-rdm and 2-rdm matrices. This workflow is preliminary and at the very least shows the calculations can be "done", whether or not the entanglement entropy values are helpful remains to be seen.

In [67]:
from openfermion.ops import FermionOperator
from openfermion.transforms import get_molecular_data

# Subsystem: choose active orbitals (e.g., 0 through 3)
#active_orbitals = [0, 1, 2, 3]

# Get the reduced density matrix over that subsystem
#from openfermion.utils import reduced_density_matrix

# Construct full RDM from 1- and 2-RDMs
rdm_sub = one_rdm


# Now compute entropy
S = von_neumann_entropy(rdm_sub, base=2)
print("Subsystem entanglement entropy:", S)


Subsystem entanglement entropy: -9.595816957284498


In [3]:
from itertools import product

A =[1]
B = [2]
C = [3]
D = [4]

cart_prod = product(A, B, C, D)
print(list(cart_prod))

[(1, 2, 3, 4)]
